In [1]:
import getpass
import os
os.environ["COHERE_API_KEY"] = getpass.getpass()

 ········


In [2]:
from langchain_cohere import ChatCohere
from langchain_community.chat_message_histories import ChatMessageHistory

In [3]:
chat_model = ChatCohere(model='command-r-plus', temperature=0.2)
history = ChatMessageHistory()

In [4]:
while (True):
    # Getting user message
    user_message = input('پیام خود را بنویسید ... (برای خروج 0 را بزنید)\n')

    # Exit the loop if the user enters '0'
    if (user_message == '0'):
        break
    else:
        print('User:',user_message, '\n')

        history.add_user_message(user_message)

        model_message = ''
        print('AI: ', end='')
        for chunk in chat_model.stream(history.messages):
            model_message = model_message+ chunk.content
            print(chunk.content, end="",flush=True)
        print('\n')

        history.add_ai_message(model_message)

پیام خود را بنویسید ... (برای خروج 0 را بزنید)
 0


# Lecture 2

In [5]:
from langchain_core.prompts import ChatPromptTemplate

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "تو یک مدل گفت‌وگو هستی که مانند دوست با کاربر صحبت می‌کنی.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat_model

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory

In [8]:
# A dictionary to store the chat history for each session id
store = {}

# A function that returns the chat history for a given session id
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # If the session id is not in the store dictionary
    if session_id not in store:
        # Create a new ChatMessageHistory object and add it to the store
        store[session_id] = ChatMessageHistory()
    # Return the message history for the session id
    return store[session_id]

In [9]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [10]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
)

In [11]:
while (True):

    # Get user message
    user_message = input('پیام خود را بنویسید ... (برای خروج 0 را بزنید)\n')

    # Exit the loop if the user enters '0'
    if (user_message == '0'):
        break
    else:
        print('User:', user_message, '\n')

        model_message = ''
        print('AI: ', end='')
        for chunk in with_message_history.stream({"input": user_message},
                                                 {"configurable": {"session_id": "1234"}}):
            model_message =  model_message + chunk.content
            print(chunk.content, end="", flush=True)
        print('\n')

پیام خود را بنویسید ... (برای خروج 0 را بزنید)
 0


# Lecture 3
## Trimming Messages

In [12]:
from langchain_core.messages import trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [13]:
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

chain_with_trimming = (
    # Replace the current chat_history with the trimmed chat history
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | chain
)

In [14]:
with_message_history = RunnableWithMessageHistory(
    chain_with_trimming,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history",
)

In [15]:
while (True):

    # Get user message
    user_message = input('پیام خود را بنویسید ... (برای خروج 0 را بزنید)\n')

    # Exit the loop if the user enters '0'
    if (user_message == '0'):
        break
    else:
        print('User:', user_message, '\n')

        model_message = ''
        print('AI: ', end='')
        for chunk in with_message_history.stream({"input": user_message},
                                                 {"configurable": {"session_id": "1234"}}):
            model_message =  model_message + chunk.content
            print(chunk.content, end="", flush=True)
        print('\n')

پیام خود را بنویسید ... (برای خروج 0 را بزنید)
 0


# Lecture 4
### Summary Memory

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "تو یک دستیار گفت‌وگو هستی. تمام سوالات را به بهترین شکل پاسخ بده. تاریخچه‌ی گفت‌وگو شامل خلاصه‌ای از مکالمه با کاربر تا این لحظه نیز در اختیارت قرار گرفته است.",
            # Or something like this in English: "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with."
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat_model

In [17]:
chat_message_history = ChatMessageHistory()

with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [19]:
chat_model_for_history = ChatCohere(model='command-r-plus', temprature=0)

In [20]:
def summarize_messages(chain_input):
    stored_messages = chat_message_history.messages
    
    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user", "پیام‌های بالا را در یک پیام خلاصه‌شده فشرده کن. تا جایی که می‌تونی تمام جزئیات و اطلاعات خاص و مهم هر پیام رو در این خلاصه نگه دار"
                # Or something like this in English: "Distill the above chat messages into a single summary message. Include as many specific details as you can."
            ),
        ]
    )
    
    print("Summarizing messages...")

    summarization_chain = summarization_prompt | chat_model_for_history

    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    print("Finished! The summary message is:", summary_message.content)

    chat_message_history.clear()

    chat_message_history.add_message(summary_message)

    return True

In [21]:
chain_with_summarization = (
    RunnablePassthrough.assign(chat_history=summarize_messages)
    | with_message_history
)

In [23]:
response = chain_with_summarization.invoke(
    {"input": "سلام چطوری؟ من متین هستم."},
    {"configurable": {"session_id": "unused"}},
)
print(response.content)

Summarizing messages...
Finished! The summary message is: دیبا با سلام و احوالپرسی شروع کرد و خودش را معرفی کرد. در پاسخ، شما خودتان را Command R+ معرفی کردید و ابراز کردید که یک دستیار گفت‌وگوی هوشمند هستید و آماده کمک کردن به دیبا هستید.
سلام متین! خوشحالم که با شما آشنا می‌شوم. من Command R+ هستم، یک دستیار گفت‌وگوی هوشمند و پیشرفته. چطور می‌تونم کمکتون کنم؟


In [24]:
response = chain_with_summarization.invoke(
    {"input": "یک جوک خنده‌دار درباره‌ی برنامه‌نویسی میگی؟"},
    {"configurable": {"session_id": "unused"}},
)
print(response.content)

Summarizing messages...
Finished! The summary message is: دیبا و متین با سلام و احوالپرسی شروع کردند و خود را معرفی نمودند. Command R+، یک دستیار گفت‌وگوی هوشمند و پیشرفته، نیز خود را معرفی کرد و آمادگی خود را برای کمک به کاربران ابراز نمود.
چرا برنامه‌نویس‌ها نمی‌تونن دوش بگیرن؟

چون به جای SOAP از DOAP استفاده می‌کنن!
